In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as mpl
from matplotlib import rc
rc('text',usetex=True)
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'svg'
% config InlineBackend.rc = {'figure.figsize': (5,3.5)}

In [13]:
import numpy as np
from numpy import sqrt
import numpy.linalg as la
import cvxpy as cvx
from helper import create_sample,max_error2,in_error,out_error,solve
from cd.model.utility import LipschitzExpUtility as leu, RiskNeutralUtility as rnu
from cd.model.distrs import RademacherDistribution
import cd.model.synth_data as synth
import scaler
from itertools import product

In [257]:
n_true = 10_000
X = RademacherDistribution().sample(n_true)
r = RademacherDistribution().sample(n_true)
t_true = X*r
t_true = t_true[:,None]

In [261]:
max_error2(t_true,rnu(),n,500)

0.19087756991779872

In [311]:
ns = np.arange(10,52,2)
us = [rnu(),leu(1)]
m = 2_000
ts = create_sample(t_true,max(ns),m)
errors = np.empty((len(ns),len(us)))
for (i,n),(j,u) in product(enumerate(ns),enumerate(us)):
    sample = ts[:,:n]
    errors[i,j] = max_error2(t_true,u,n,m,sample)

Combined backtracking failed 86 0 4 0 sigma 0.0001


Combined backtracking failed 69 0 21 0 sigma 1


Combined backtracking failed 90 0 0 0 sigma 0.0001


Combined backtracking failed 90 0 0 0 sigma 0.80702


Combined backtracking failed 90 0 0 0 sigma 0.986126


In [312]:
plt.plot(ns,1/ns+2*np.sqrt(2*np.log(1/0.05)/ns),':',label="Borne th\\'eorique")
plt.plot(ns,errors[:,0],'k--',label='$\\mu = \\infty$')
plt.plot(ns,errors[:,1],'C3',label='$\\mu = 1$')
plt.legend()
# plt.yscale('sqrtscale')
plt.xlabel('$n$')
plt.ylabel("Erreur de g\\'eneralisation")
# plt.savefig('fig/raderr.pdf',bbox_inches='tight',pad_inches=0)

In [7]:
n_true = 10_000
p = 2
X = [RademacherDistribution() for _ in range(p)]
R = RademacherDistribution()
M = synth.GaussianMarket(X,R)
X,r = M.sample(n_true)
t_true = X*r[:,None]
q_true = t_true.mean(axis=0)


In [27]:
p = 2
Xs = [RademacherDistribution() for _ in range(p)]
R = RademacherDistribution()
M = synth.GaussianMarket(X,R,bias=False)

In [30]:
Xs

In [34]:
M.Xs

array([[ 1., -1.,  1.],
       [ 1., -1., -1.],
       [ 1., -1.,  1.],
       ..., 
       [ 1.,  1., -1.],
       [ 1., -1.,  1.],
       [ 1.,  1.,  1.]])

In [9]:
qhl = la.norm(q_true)
qhl

0.46849755602350796

In [10]:
ts = create_sample(t_true,20,5)

In [14]:
qs = solve(ts)

In [20]:
in_error(rnu(),ts,qs),out_error(rnu(),t_true,qs)

(array([ 0.40999855,  0.39999844,  0.44999882,  0.75999945,  0.21000121]),
 array([ 0.26633906,  0.26703896,  0.29999921,  0.26707981,  0.19906115]))

In [21]:
la.norm(qs,axis=1)**2,qs@q_true

(array([ 0.40999709,  0.39999689,  0.44999764,  0.7599989 ,  0.21000242]),
 array([ 0.26633906,  0.26703896,  0.29999921,  0.26707981,  0.19906115]))

In [26]:
d = {'a':2}
'b' in d
d['a']

2

In [22]:
'a' in {'a':2}

True

In [391]:
ns = np.arange(2,150,5)
u = rnu()
m = 5000
ts = create_sample(t_true,max(ns),m)
maxnorm = np.empty(len(ns))
mnerror = np.empty(len(ns))
for i,n in enumerate(ns):
    samples = ts[:,:n]
    # samples = create_sample(t_true,n,m)
    qs = solve(samples,lamb=1/2,u=u)
    maxnorm[i] = np.max(la.norm(qs,axis=1))

In [399]:
plt.plot(ns,qhl+2*np.sqrt(3*np.log(1/0.05)/(2*ns)),':',label="Borne th\\'eorique $(\\delta=0.05)$")
plt.plot(ns,maxnorm,'C3',label='$\max\|\mathcal{Q}(\mathcal{S}_n)\|$')
plt.plot(ns,0*ns+qhl,'k--',label='\|q^\star_\lambda\|')
plt.axis(ymin=0.1)
plt.xlabel('$n$')
plt.yticks([])
plt.legend()
# plt.savefig('fig/normqhqsl.pdf',bbox_inches='tight',pad_inches=0)

In [558]:
truerror = out_error(u,t_true,qtrue)

In [471]:
ns = np.arange(10,52,2)
# us = [rnu(),leu(1)]
us = [rnu()]
m = 5_000
ts = create_sample(t_true,max(ns),m)
errors = np.empty((len(ns),len(us)))
for (i,n),(j,u) in product(enumerate(ns),enumerate(us)):
    sample = ts[:,:n]
    errors[i,j] = max_error2(t_true,u,n,m,sample)

In [634]:
?np.percentile

In [0]:
ns = np.arange(10,2002,2)
m = 2_000
ts = create_sample(t_true,max(ns),m)
maxerror = np.empty(len(ns))
innorm = np.empty(len(ns))
mnerror = np.empty(len(ns))
for i,n in enumerate(ns):
    sample = ts[:,:n]
    qs = solve(ts,u=u)
    inerror = in_error(u,sample,qs)
    outerror = out_error(u,t_true,qs)
    mnerror[i] = np.mean(np.abs(inerror - outerror))
    # maxerror[i] = np.max(inerror - outerror)
    maxerror[i] = np.percentile(inerror - outerror,99)
    # innorm[i] = la.norm(qs,axis=1)

    

In [0]:
tol = np.log(1/0.05)
# plt.plot(ns,6*(1/n+3*np.sqrt(np.log(1/0.05)/(2*ns))),':',label="Borne th\\'eorique")
# plt.plot(ns,-truerror + 6*((ns**(-1/2) + ns**(-1.) + ns**(-3/2)) * np.sqrt(np.log(1/0.05)/2)))
# plt.plot(ns,truerror+sqrt(tol/(2*ns)))
plt.plot(ns,maxerror,label='max')
plt.plot(ns,mnerror,label='mean')
plt.plot(ns,ns*0 + truerror,label='truerror')
plt.axis(ymin=0)
# plt.yscale('sqrtscale')
plt.legend()

In [472]:
plt.plot(ns,3/ns+12*np.sqrt(np.log(1/0.05)/(2*ns)),':',label="Borne th\\'eorique")
plt.plot(ns,errors,'k--',label='$\\mu = \\infty$')
# plt.plot(ns,errors[:,1],'C3',label='$\\mu = 1$')
plt.legend()
# plt.yscale('sqrtscale')
plt.xlabel('$n$')
plt.ylabel("Erreur de g\\'eneralisation")
plt.axis(ymin=0)
# plt.yscale('sqrtscale')
# plt.savefig('fig/raderr.pdf',bbox_inches='tight',pad_inches=0)

(8.0, 52.0, 0, 5.1702191258582362)